# Understanding word vectors

... for, like, actual poets. By [Allison Parrish](http://www.decontextualize.com/)

In this tutorial, I'm going to show you how word vectors work.

In [ ]:
pip install pandas

In [ ]:
pip install numpy

In [ ]:
pip install matplotlib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 25)
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (10, 4)



## Why word vectors for poetry?

Tzara proposed a method of composing a Dada poem: cut out the words of a text, shake them in a bag, then write down the words as you remove them at random from the bag. The very idea caused a riot and sundered the avant garde in twain (or so the story goes). For poets, word vectors are (for better or worse) a tool to help soften the blow of cut-up techniques: instead of selecting words at random, we might select units of text that are *close in meaning* to other units. This can yield poetic juxtapositions with subtle effects impossible to achieve with other techniques.

Also, it's fun!

## Animal similarity and simple linear algebra

We'll begin by considering a small subset of English: words for animals. Our task is to be able to write computer programs to find similarities among these words and the creatures they designate. To do this, we might start by making a spreadsheet of some animals and their characteristics. In Python, you'd define such a spreadsheet like this:

In [ ]:
animals = [
    {'name': 'kitten', 'cuteness': 95, 'size': 15},
    {'name': 'hamster', 'cuteness': 80, 'size': 8},
    {'name': 'tarantula', 'cuteness': 8, 'size': 3},
    {'name': 'puppy', 'cuteness': 90, 'size': 20},
    {'name': 'crocodile', 'cuteness': 5, 'size': 40},
    {'name': 'dolphin', 'cuteness': 60, 'size': 45},
    {'name': 'panda bear', 'cuteness': 75, 'size': 40},
    {'name': 'lobster', 'cuteness': 2, 'size': 15},
    {'name': 'capybara', 'cuteness': 70, 'size': 30},
    {'name': 'elephant', 'cuteness': 65, 'size': 90},
    {'name': 'mosquito', 'cuteness': 1, 'size': 1},
    {'name': 'goldfish', 'cuteness': 25, 'size': 2},
    {'name': 'horse', 'cuteness': 50, 'size': 50},
    {'name': 'chicken', 'cuteness': 25, 'size': 15}
]
animal_lookup = {item['name']: (item['cuteness'], item['size']) for item in animals}

And then display it:

In [ ]:
pd.DataFrame(animals, columns=['name', 'cuteness', 'size'])

This table associates a handful of animals with two numbers: their cuteness and their size, both in a range from zero to one hundred. (The values themselves are simply based on my own judgment. Your taste in cuteness and evaluation of size may differ significantly from mine. As with all data, these data are simply a mirror reflection of the person who collected them.)

These values give us everything we need to make determinations about which animals are similar (at least, similar in the properties that we've included in the data). Try to answer the following question: Which animal is most similar to a capybara? You could go through the values one by one and do the math to make that evaluation, but visualizing the data as points in 2-dimensional space makes finding the answer very intuitive:

In [ ]:
plt.figure(figsize=(8, 8))
plt.scatter([item[0] for item in animal_lookup.values()],
            [item[1] for item in animal_lookup.values()])
plt.xlabel('cuteness')
plt.ylabel('size')
for label, (cute, size) in animal_lookup.items():
    plt.text(cute+1, size+1, label, fontsize=12)
plt.show()

The plot shows us that the closest animal to the capybara is the panda bear (again, in terms of its subjective size and cuteness). One way of calculating how "far apart" two points are is to find their *Euclidean distance*. (This is simply the length of the line that connects the two points.) For points in two dimensions, Euclidean distance can be calculated with the following Python function:

In [ ]:
import math
def distance2d(a, b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

(The `**` operator raises the value on its left to the power on its right.)

So, the distance between "capybara" (70, 30) and "panda" (74, 40):

In [ ]:
distance2d(animal_lookup['capybara'], animal_lookup['panda bear']) # panda and capybara

... is less than the distance between "tarantula" and "elephant":

In [ ]:
distance2d(animal_lookup['tarantula'], animal_lookup['elephant']) # tarantula and elephant

Modeling animals in this way has a few other interesting properties. For example, you can pick an arbitrary point in "animal space" and then find the animal closest to that point. If you imagine an animal of size 25 and cuteness 30, you can easily look at the space to find the animal that most closely fits that description: the chicken.

Reasoning visually, you can also answer questions like: what's halfway between a chicken and an elephant? Simply draw a line from "elephant" to "chicken," mark off the midpoint and find the closest animal. (According to our chart, halfway between an elephant and a chicken is a horse.)

You can also ask: what's the *difference* between a hamster and a tarantula? According to our plot, it's about seventy five units of cute (and a few units of size).

The relationship of "difference" is an interesting one, because it allows us to reason about *analogous* relationships. In the chart below, I've drawn an arrow from "tarantula" to "hamster" (in blue):

In [ ]:
plt.figure(figsize=(8, 8))
plt.scatter([item[0] for item in animal_lookup.values()],
            [item[1] for item in animal_lookup.values()])
plt.xlabel('cuteness')
plt.ylabel('size')
for label, (cute, size) in animal_lookup.items():
    plt.text(cute+1, size+1, label, fontsize=12)
plt.arrow(
    *(animal_lookup['tarantula']),
    *(np.array(animal_lookup['hamster']) - np.array(animal_lookup['tarantula'])),
    fc="b", ec="b", head_width=1.5, head_length=2, linewidth=1.5)
plt.arrow(
    *(animal_lookup['chicken']),
    *(np.array(animal_lookup['hamster']) - np.array(animal_lookup['tarantula'])),
    fc="r", ec="r", head_width=1.5, head_length=2, linewidth=1.5)
plt.show()

You can understand this arrow as being the *relationship* between a tarantula and a hamster, in terms of their size and cuteness (i.e., hamsters and tarantulas are about the same size, but hamsters are much cuter). In the same diagram, I've also transposed this same arrow (this time in red) so that its origin point is "chicken." The arrow ends closest to "kitten." What we've discovered is that the animal that is about the same size as a chicken but much cuter is... a kitten. To put it in terms of an analogy:

    Tarantulas are to hamsters as chickens are to kittens.
    
A sequence of numbers used to identify a point is called a *vector*, and the kind of math we've been doing so far is called *linear algebra.* (Linear algebra is surprisingly useful across many domains: It's the same kind of math you might do to, e.g., simulate the velocity and acceleration of a sprite in a video game.)

A set of vectors that are all part of the same data set is often called a *vector space*. The vector space of animals in this section has two *dimensions*, by which I mean that each vector in the space has two numbers associated with it (i.e., two columns in the spreadsheet). The fact that this space has two dimensions just happens to make it easy to *visualize* the space by drawing a 2D plot. But most vector spaces you'll work with will have more than two dimensions—sometimes many hundreds. In those cases, it's more difficult to visualize the "space," but the math works pretty much the same.

## Language with vectors: colors

So far, so good. We have a system in place—albeit highly subjective—for talking about animals and the words used to name them. I want to talk about another vector space that has to do with language: the vector space of colors.

Colors are often represented in computers as vectors with three dimensions: red, green, and blue. Just as with the animals in the previous section, we can use these vectors to answer questions like: which colors are similar? What's the most likely color name for an arbitrarily chosen set of values for red, green and blue? Given the names of two colors, what's the name of those colors' "average"?

We'll be working with this [color data](https://github.com/dariusk/corpora/blob/master/data/colors/xkcd.json) from the [xkcd color survey](https://blog.xkcd.com/2010/05/03/color-survey-results/). The data relates a color name to the RGB value associated with that color. [Here's a page that shows what the colors look like](https://xkcd.com/color/rgb/). Download the color data and put it in the same directory as this notebook.

A few notes before we proceed:

* We're using `numpy` arrays below to implement vector arithmetic. You'll need to install `numpy` to get the examples to work. (If you used Anaconda to install Python, you have `numpy` already.) [Here's a quick introduction on how to use numpy for vector arithmetic.](https://docs.scipy.org/doc/numpy/user/quickstart.html)
* If you're interested in perceptually accurate color math in Python, consider using the [colormath library](http://python-colormath.readthedocs.io/en/latest/).

Now, import the `json` library and load the color data:

In [ ]:
import json

In [ ]:
color_data = json.loads(open("xkcd.json").read())

The following function converts colors from hex format (`#1a2b3c`) to a tuple of integers:

In [ ]:
def hex_to_int(s):
    s = s.lstrip("#")
    return np.array([int(s[:2], 16), int(s[2:4], 16), int(s[4:6], 16)])

And the following cell creates a dictionary and populates it with mappings from color names to RGB vectors for each color in the data:

In [ ]:
colors = dict()
for item in color_data['colors']:
    colors[item["color"]] = hex_to_int(item["hex"])

Testing it out:

In [ ]:
colors['olive']

In [ ]:
colors['red']

In [ ]:
colors['black']

In [ ]:
colors['cyan']

### Vector math

Before we keep going, we'll need some functions for performing basic vector "arithmetic." These functions will work with vectors in spaces of any number of dimensions.

The first function returns the Euclidean distance between two points:

In [ ]:
from numpy.linalg import norm
def distance(a, b):
    return norm(a - b)

In [ ]:
distance(colors['cyan'], colors['blue'])

In [ ]:
distance(np.array([10, 1]), np.array([5, 2]))

Subtracting vectors:

In [ ]:
colors['cyan'] - colors['blue']

Adding vectors:

In [ ]:
colors['cyan'] + colors['blue']

You can find the average of two vectors using the expected formula:

In [ ]:
(colors['cyan'] + colors['blue']) / 2

Or use the following function, which finds the mean of any number of vectors:

In [ ]:
def meanv(vecs):
    total = np.sum(vecs, axis=0)
    return total / len(vecs)

In [ ]:
meanv([colors['red'], colors['pink'], colors['maroon']])

Just as a test, the following cell shows that the distance from "red" to "green" is greater than the distance from "red" to "pink":

In [ ]:
distance(colors['red'], colors['green']) > distance(colors['red'], colors['pink'])

### Finding the closest item

Just as we wanted to find the animal that most closely matched an arbitrary point in cuteness/size space, we'll want to find the closest color name to an arbitrary point in RGB space. The easiest way to find the closest item to an arbitrary vector is simply to find the distance between the target vector and each item in the space, in turn, then sort the list from closest to most distant.

Calculating the distance between two points, however, is computationally expensive, especially when you're working with data that has many dimensions. To solve this problem, computer scientists and mathematicians came up with the idea of [approximate nearest neighbor search](https://en.wikipedia.org/wiki/Nearest_neighbor_search#Approximation_methods), a technique for finding similar points in high-dimensional spaces that make use of various tricks to speed up the process (potentially at the cost of accuracy).

We're going to use a library I made called [Simple Neighbors](https://github.com/aparrish/simpleneighbors) that builds such an approximate nearest neighbors index to quickly return the closest items for any given vector. (Simple Neighbors is based on [Annoy](https://pypi.python.org/pypi/annoy).)

Install Simple Neighbors like so:

In [ ]:
import sys
!{sys.executable} -m pip install simpleneighbors

You'll want to install Annoy as well, to speed up the nearest neighbor search. As of this writing, I recommend Annoy 1.16.3:

In [ ]:
import sys
!{sys.executable} -m pip install annoy==1.16.3

If you get an error from the above, and you're using Anaconda, you can try installing the Anaconda package:

In [ ]:
import sys
!conda install -y --prefix {sys.prefix} -c conda-forge python-annoy 

If neither of those works—especially if you're using Windows—you may need to install a C++ compiler, or you can use this notebook on Binder. (If you're a student, come see me for more info.)

Once you have the library installed, import it like so:

In [ ]:
from simpleneighbors import SimpleNeighbors

The first parameter to `SimpleNeighbors()` is the number of dimensions in the data; the second is the distance metric to use. (This defaults to cosine distance, but in this case we want to use Euclidean distance.)

The `.add_one()` method adds an item and a vector to the index. Once all the items are added, `.build()` actually builds the index. This should go quick, since we don't have that much data!

In [ ]:
color_lookup = SimpleNeighbors(3, 'euclidean')
for name, vec in colors.items():
    color_lookup.add_one(name, vec)
color_lookup.build()

To find the nearest item to a specified vector, pass the vector to the `.nearest()` method:

In [ ]:
color_lookup.nearest(colors['red'])

Limit the number of results returned with the second parameter:

In [ ]:
color_lookup.nearest(colors['red'], 3)

Here are the colors closest to (150, 60, 150):

In [ ]:
color_lookup.nearest([150, 60, 150])

The `.dist()` method gives the distance between two items in the index:

In [ ]:
color_lookup.dist('rose', 'pink')

In [ ]:
color_lookup.dist('green', 'purple')

And you can check the `.corpus` attribute to see if an item is even in the index to begin with:

In [ ]:
'orange' in color_lookup.corpus

In [ ]:
'kitten' in color_lookup.corpus

### Color magic

The magical part of representing words as vectors is that the vector operations we defined earlier appear to operate on language the same way they operate on numbers. For example, if we find the word closest to the vector resulting from subtracting "red" from "purple," we get a series of "blue" colors:

In [ ]:
color_lookup.nearest(colors['purple'] - colors['red'])

This matches our intuition about RGB colors, which is that purple is a combination of red and blue. Take away the red, and blue is all you have left.

You can do something similar with addition. What's blue plus green?

In [ ]:
color_lookup.nearest(colors['blue'] + colors['green'])

That's right, it's something like turquoise or cyan! What if we find the average of black and white? Predictably, we get gray:

In [ ]:
# the average of black and white: medium grey
color_lookup.nearest(meanv([colors['white'], colors['black']]))

Just as with the tarantula/hamster example from the previous section, we can use color vectors to reason about relationships between colors. In the cell below, finding the difference between "pink" and "red" then adding it to "blue" seems to give us a list of colors that are to blue what pink is to red (i.e., a slightly lighter, less saturated shade):

In [ ]:
# an analogy: pink is to red as X is to blue
pink_to_red = colors['pink'] - colors['red']
color_lookup.nearest(pink_to_red + colors['blue'])

Another example of color analogies: Navy is to blue as true green/dark grass green is to green:

In [ ]:
# another example: 
navy_to_blue = colors['navy'] - colors['blue']
color_lookup.nearest(navy_to_blue + colors['green'])

The examples above are fairly simple from a mathematical perspective but nevertheless *feel* magical: they're demonstrating that it's possible to use math to reason about how people use language.

### Interlude: A Love Poem That Loses Its Way

In [ ]:
import random
red = colors['red']
blue = colors['blue']
for i in range(14):
    rednames = color_lookup.nearest(red)
    bluenames = color_lookup.nearest(blue)
    print("Roses are " + rednames[0] + ", violets are " + bluenames[0])
    if len(rednames) > 1:
      red = colors[random.choice(rednames[1:])]
    else:
      red = colors[rednames[0]]

    if len(bluenames) > 1:
      blue = colors[random.choice(bluenames[1:])]
    else:
       blue = colors[bluenames[0]]


### Doing bad digital humanities with color vectors

With the tools above in hand, we can start using our vectorized knowledge of language toward academic ends. In the following example, I'm going to calculate the average color of Mary Shelley's *Frankenstein*.

(Before you proceed, make sure to [download the text file from Project Gutenberg](http://www.gutenberg.org/files/84/84-0.txt) and place it in the same directory as this notebook.)

First, we'll load [spaCy](https://spacy.io/). Note: For the rest of this tutorial to work, you'll want to download at least the medium model for English. The default "small" model doesn't include word vectors. I've also written an [introduction to spaCy](nlp-concepts-with-spacy.ipynb) that includes installation instructions.

In [ ]:
import spacy
nlp = spacy.load('en_core_web_md')

To calculate the average color, we'll follow these steps:

1. Parse the text into words
2. Check every word to see if it names a color in our vector space. If it does, add it to a list of vectors.
3. Find the average of that list of vectors.
4. Find the color(s) closest to that average vector.

The following cell performs steps 1-3:

In [ ]:
doc = nlp(open("84-0.txt").read())
# use word.lower_ to normalize case
drac_colors = [colors[word.lower_] for word in doc if word.lower_ in colors]
avg_color = meanv(drac_colors)
print(avg_color)

Now, we'll pass the averaged color vector to the `closest()` function, yielding... well, it's just a grey mush, which is kinda what you'd expect from adding a bunch of colors together willy-nilly.

In [ ]:
color_lookup.nearest(avg_color)

On the other hand, here's what we get when we average the colors of Charlotte Perkins Gilman's classic *The Yellow Wallpaper*. ([Download from here](http://www.gutenberg.org/cache/epub/1952/pg1952.txt) and save in the same directory as this notebook if you want to follow along.) The result definitely reflects the content of the story, so maybe we're on to something here.

In [ ]:
doc = nlp(open("1952-0.txt").read())
wallpaper_colors = [colors[word.lower_] for word in doc if word.lower_ in colors]
avg_color = meanv(wallpaper_colors)
color_lookup.nearest(avg_color)

Exercise for the reader: Use the vector arithmetic functions to rewrite a text, making it...

* more blue (i.e., add `colors['blue']` to each occurrence of a color word); or
* more light (i.e., add `colors['white']` to each occurrence of a color word); or
* darker (i.e., attenuate each color. You might need to write a vector multiplication function to do this one right.)

## Distributional semantics

In the previous section, the examples are interesting because of a simple fact: colors that we think of as similar are "closer" to each other in RGB vector space. In our color vector space, or in our animal cuteness/size space, you can think of the words identified by vectors close to each other as being *synonyms*, in a sense: they sort of "mean" the same thing. They're also, for many purposes, *functionally identical*. Think of this in terms of writing, say, a search engine. If someone searches for "mauve trousers," then it's probably also okay to show them results for, say,

In [ ]:
for cname in color_lookup.nearest(colors['mauve']):
    print(cname + " trousers")

That's all well and good for color words, which intuitively seem to exist in a multidimensional continuum of perception, and for our animal space, where we've written out the vectors ahead of time. But what about... arbitrary words? Is it possible to create a vector space for all English words that has this same "closer in space is closer in meaning" property?

To answer that, we have to back up a bit and ask the question: what does *meaning* mean? No one really knows, but one theory popular among computational linguists, computer scientists and other people who make search engines is the [Distributional Hypothesis](https://en.wikipedia.org/wiki/Distributional_semantics), which states that:

    Linguistic items with similar distributions have similar meanings.
    
What's meant by "similar distributions" is *similar contexts*. Take for example the following sentences:

    It was really cold yesterday.
    It will be really warm today, though.
    It'll be really hot tomorrow!
    Will it be really cool Tuesday?
    
According to the Distributional Hypothesis, the words `cold`, `warm`, `hot` and `cool` must be related in some way (i.e., be close in meaning) because they occur in a similar context, i.e., between the word "really" and a word indicating a particular day. (Likewise, the words `yesterday`, `today`, `tomorrow` and `Tuesday` must be related, since they occur in the context of a word indicating a temperature.)

In other words, according to the Distributional Hypothesis, a word's meaning is just a big list of all the contexts it occurs in. Two words are closer in meaning if they share contexts.

## Word vectors by counting contexts

So how do we turn this insight from the Distributional Hypothesis into a system for creating general-purpose vectors that capture the meaning of words? Maybe you can see where I'm going with this. What if we made a *really big* spreadsheet that had one column for every context for every word in a given source text. Let's use a small source text to begin with, such as this excerpt from Dickens:

    It was the best of times, it was the worst of times.

The code in the following cell builds a table of what such a spreadsheet might look like. (Don't worry about understanding this code! But feel free to play around with it.)

In [ ]:
from collections import defaultdict
src = "it was the best of times it was the worst of times"
tokens = ['START'] + src.split() + ['END']
contexts = defaultdict(lambda: defaultdict(int))
tok_types = {}
for i in range(len(tokens)-2):
    tok_types[tokens[i+1]] = 1
    contexts[(tokens[i], tokens[i+2])][tokens[i+1]] += 1
data = {" ___ ".join(k): [v[tok] for tok in tok_types.keys()] for k, v in contexts.items()}
df = pd.DataFrame(data=data, index=tok_types)
df

The spreadsheet has one column for every possible context, and one row for every word. The values in each cell correspond with how many times the word occurs in the given context. The numbers in the columns constitute that word's vector, i.e., the vector for the word `of` is

    [0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
    
Because there are ten possible contexts, this is a ten dimensional space! It might be strange to think of it, but you can do vector arithmetic on vectors with ten dimensions just as easily as you can on vectors with two or three dimensions, and you could use the same distance formula that we defined earlier to get useful information about which vectors in this space are similar to each other. In particular, the vectors for `best` and `worst` are actually the same (a distance of zero), since they occur only in the same context (`the ___ of`):

    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
    
Of course, the conventional way of thinking about "best" and "worst" is that they're *antonyms*, not *synonyms*. But they're also clearly two words of the same kind, with related meanings (through opposition), a fact that is captured by this distributional model.

### Contexts and dimensionality

Of course, in a corpus of any reasonable size, there will be many thousands if not many millions of possible contexts. It's difficult enough working with a vector space of ten dimensions, let alone a vector space of a million dimensions! It turns out, though, that many of the dimensions end up being superfluous and can either be eliminated or combined with other dimensions without significantly affecting the predictive power of the resulting vectors. The process of getting rid of superfluous dimensions in a vector space is called [dimensionality reduction](https://en.wikipedia.org/wiki/Dimensionality_reduction), and most implementations of count-based word vectors make use of dimensionality reduction so that the resulting vector space has a reasonable number of dimensions (say, 100—300, depending on the corpus and application).

The question of how to identify a "context" is itself very difficult to answer. In the toy example above, we've said that a "context" is just the word that precedes and the word that follows. Depending on your implementation of this procedure, though, you might want a context with a bigger "window" (e.g., two words before and after), or a non-contiguous window (skip a word before and after the given word). You might exclude certain "function" words like "the" and "of" when determining a word's context, or you might [lemmatize](https://en.wikipedia.org/wiki/Lemmatisation) the words before you begin your analysis, so two occurrences with different "forms" of the same word count as the same context. These are all questions open to research and debate, and different implementations of procedures for creating count-based word vectors make different decisions on this issue.

### GloVe vectors

But you don't have to create your own word vectors from scratch! Many researchers have made downloadable databases of pre-trained vectors. One such project is Stanford's [Global Vectors for Word Representation (GloVe)](https://nlp.stanford.edu/projects/glove/). These 300-dimensional vectors are included with spaCy, and they're the vectors we'll be using for the rest of this tutorial.

## Word vectors in spaCy

Okay, let's have some fun with real word vectors. We're going to use the GloVe vectors that come with spaCy to creatively analyze and manipulate the text of *Frankenstein*. First, make sure you've got `spacy` imported:

In [ ]:
import spacy

The following cell loads the language model:

In [ ]:
nlp = spacy.load('en_core_web_md')

You can see the vector of any word in spaCy's vocabulary using the `vocab` attribute, like so:

In [ ]:
nlp.vocab['kitten'].vector

spaCy's English model uses 300-dimensional pre-trained GloVe vectors.

For the sake of convenience, the following function gets the vector of a given string from spaCy's vocabulary:

In [ ]:
def vec(s):
    return nlp.vocab[s].vector

## Filter by word frequency

In just a second, I'm going to show you how to make a nearest-neighbors lookup to find words that are semantically similar. We'll use the vectors from spaCy, but there's a problem—spaCy's database of words is very large. You can find out how many words are in spaCy's database by accessing the `.meta` attribute of the language object:

In [ ]:
nlp.meta['vectors']

As of this writing, there are 20k vectors in the database, but over 680k individual tokens. (Many tokens don't have vectors; some tokens are mapped to the same vector.) That's too many tokens for our nearest-neighbor lookup to be reasonably fast. Also, because the tokens are drawn from the corpus that spaCy's model is trained on, many of the tokens are very uncommon words or misspellings of other words. We probably don't want to use this for text replacement tasks, which is what we've got an eye on doing in the examples below.

The best way to fix this problem is to filter the words before we put them into the nearest-neighbor lookup. We're going to do this by *token frequency*, i.e., how frequently that word appears in English. A word's frequency is calculated as the number of times that word occurs in a corpus, divided by the total number of words in the corpus. The [wordfreq](https://github.com/LuminosoInsight/wordfreq) Python package has a good database of word frequencies in many different languages. To simplify things for this notebook, I've prepared [a JSON file with the top 25k English words, along with their probabilities](https://raw.githubusercontent.com/aparrish/wordfreq-en-25000/main/wordfreq-en-25000-log.json). Before you continue, download that file into the same directory as this notebook, or with `curl` like so:

In [ ]:
!curl -L -O https://raw.githubusercontent.com/aparrish/wordfreq-en-25000/main/wordfreq-en-25000-log.json

The code in the following cell loads this data into a dictionary that we can use to look up the probability of a given word:

In [ ]:
import json
prob_lookup = dict(json.load(open("./wordfreq-en-25000-log.json")))

Now you can look up a word's probability with the `prob_lookup` dictionary. As you might imagine, the frequency of most words is very small, meaning that if we stored the frequency as a decimal fraction (e.g., 0.00006), the data on disk would mostly consist of the digit `0`. To make the numbers easier to store and work with, the log of the word probabilities, rather than the probability itself:

In [ ]:
prob_lookup['me']

Log probabilities are always negative; the closer to zero, the more frequent the token. To get the original probability number, use `math.exp()`:

In [ ]:
import math
math.exp(prob_lookup['me'])

(You can interpret the result of the above expression to mean that the word `me` occurs about 33 times in every ten thousand words.)

Note that all of the tokens in this database are stored in lower case, and you'll get a `KeyError` for words that are not present in the database! Some other examples of using the lookup:

In [ ]:
prob_lookup['allison']

Cats aren't more frequent than dogs:

In [ ]:
prob_lookup['cats'] > prob_lookup['dogs']

Unknown tokens raise `KeyError`:

In [ ]:
prob_lookup['asdfasdf']

But you can use the `dict` object's `.get()` method to return a somewhat reasonable estimate of a word's probability, even if it's absent:

In [ ]:
prob_lookup.get('asdfasdf', -20.0)

## Looking up synonyms

So now we can finally make our synonym lookup! Here I make a `SimpleNeighbors` index that loads in all words from spaCy's vocab that (a) have an associated vector and (b) are in our database of the top 25k words.

In [ ]:
lookup = SimpleNeighbors(300)
for word in prob_lookup.keys():
    if nlp.vocab[word].has_vector:
        lookup.add_one(word, vec(word))
lookup.build()

This leaves us with nearly 25k words in the lookup (a few of the words from the word frequency list don't have vectors in spaCy, apparently):

In [ ]:
len(lookup)

Now we can get synonyms for words by looking up the word's vector and finding the nearest word in the index:

In [ ]:
lookup.nearest(vec('basketball'))

> Exercise: Limit the synonym lookup to words with a frequency greater than a particular threshold.

### Fun with spaCy and vector arithmetic

Now we can start doing vector arithmetic and finding the closest words to the resulting vectors. For example, what word is closest to the halfway point between day and night?

In [ ]:
# halfway between day and night
lookup.nearest(meanv([vec("day"), vec("night")]))

Variations of `night` and `day` are still closest, but after that we get words like `evening` and `morning`, which are indeed halfway between day and night!

Here are the closest words in *Frankenstein* to "wine":

In [ ]:
lookup.nearest(vec("wine"))

If you subtract "alcohol" from "wine" and find the closest words to the resulting vector, you're left with the superlatives you might use to describe wine:

In [ ]:
lookup.nearest(vec("wine") - vec("alcohol"))

The closest words to "water":

In [ ]:
lookup.nearest(vec("water"))

But if you add "boil" to "water," you get "simmer" and "steamed":

In [ ]:
lookup.nearest(vec("water") + vec("boil"))

## Replace with synonym

The following example replaces all nouns, verbs and adjectives with a closely-related word from the synonym lookup.

In [ ]:
frost_doc = nlp(open("frost.txt").read())

In [ ]:
output = []
for word in frost_doc:
    if word.is_alpha and word.pos_ in ('NOUN', 'VERB', 'ADJ'):
        new_word = random.choice(lookup.nearest(word.vector, 3))
        output.append(new_word)
    else:
        output.append(word.text)
    output.append(word.whitespace_)
print(''.join(output))

## Tinting meaning

You can "tint" meaning with word vectors as well. The methodology demonstrated below is to take the word vector for every noun, verb and adjective in the text, and then look up the word closest to a weighted average between the original word and a target word. (You can control the weight of the average using the `factor` variable below.) On balance, this leads to word replacements that still at least somewhat "make sense" in the original context, but have a bit of meaning from the target word.

In [ ]:
target_word = 'spaceship'
factor = 0.5

In [ ]:
output = []
for word in frost_doc:
    if word.is_alpha and word.pos_ in ('NOUN', 'VERB', 'ADJ'):
        new_word = random.choice(
            lookup.nearest((word.vector*(1-factor)) + (vec(target_word)*factor), 5))
        output.append(new_word)
    else:
        output.append(word.text)
    output.append(word.whitespace_)
print(''.join(output))

## Semantic similarity search

Every spaCy span (e.g., a document or a sentence) has a `.vector` attribute that gives a "summary" vector for the span in question. (By default, this is calculated by averaging together all of the vectors.)

In [ ]:
sent = nlp("Programming computers is fun!")

In [ ]:
sent.vector

Interestingly, we can find the single word closest in meaning to this sequence of words:

In [ ]:
lookup.nearest(sent.vector)

Or, we can use this as a sort of rudimentary search engine, making it possible to find sentences in a text that are close in meaning to any arbitrary sentence we type in. Here's how to do that! First, we'll get the list of sentences:

In [ ]:
doc = nlp(open("./84-0.txt").read())

Then, we'll create a nearest neighbors lookup with the vectors for each sentence:

In [ ]:
sentence_lookup = SimpleNeighbors(300)
for sent in doc.sents:
    # replace linebreaks to make the output a bit more neat
    sentence_lookup.add_one(sent.text.replace("\n", " "), sent.vector)
sentence_lookup.build()

Now we can find similar sentences like so:

In [ ]:
sentence_lookup.nearest(nlp("My favorite food is strawberry ice cream.").vector)

We could do something similar with spaCy noun chunks:

In [ ]:
chunk_lookup = SimpleNeighbors(300)
for chunk in doc.noun_chunks:
    chunk_text = chunk.text.replace("\n", " ")
    if chunk_text not in chunk_lookup.corpus:
        chunk_lookup.add_one(chunk_text, chunk.vector)
chunk_lookup.build()

In [ ]:
chunk_lookup.nearest(nlp("angry birds").vector)

Or even lookups for each part of speech:

In [ ]:
adj_lookup = SimpleNeighbors(300)
for word in doc:
    # .corpus of the lookup lets us determine if the word has already been added
    if word.tag_ == 'JJ' and word.text.lower() not in adj_lookup.corpus:
        adj_lookup.add_one(word.text.lower(), word.vector)
adj_lookup.build()

In [ ]:
adj_lookup.nearest(vec("happy"))

### Rewriting with parts of speech from another text

One last example and we'll call it a day. The following code uses the noun chunks and adjective lookups that I made above to rewrite one text (Frost's *The Road Not Taken*) with words and phrases from Frankenstein.

In [ ]:
frost_doc = nlp(open("frost.txt").read())

In [ ]:
output = []
for word in frost_doc:
    if word.is_alpha and word.pos_ == 'NOUN':
        new_word = random.choice(chunk_lookup.nearest(word.vector, 5))
        output.append(new_word)
    elif word.is_alpha and word.tag_ == 'JJ':
        new_word = random.choice(adj_lookup.nearest(word.vector, 5))
        output.append(new_word)
    else:
        output.append(word.text)
    output.append(word.whitespace_)
print(''.join(output))

## Beyond word vectors

The core assumptions of word vectors as a technology are that (A) words are a meaningful unit of language, and (b) words can be said to have stable "meanings" across texts. Neither of these assertions are true, but (for better or worse) they reflect beliefs that many people hold about how language works. The fact that word vectors reflect these beliefs, in my opinion, makes text analysis and text generation based on word vectors more intuitive than  the alternatives.

More recently, the task of finding semantic similarity between passages of text is accomplished using systems that operate not on words as units, but on units determined by unsupervised tokenization algorithms like [SentencePiece](https://github.com/google/sentencepiece). These systems often use neural network machine learning models (like [transformers](https://en.wikipedia.org/wiki/Transformer_(machine_learning_model))) to calculate vectors for words and longer stretches of text that take the surrounding textual context into account. These embeddings are often more accurate, at the cost of explainability and increased use of computational resources. Here are a few resources for learning more about this method:

* Google's [Universal Sentence Encoder](https://www.tensorflow.org/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder)
* The [sentence-transformers](https://github.com/UKPLab/sentence-transformers) library for Python
* [Awesome Sentence Embedding](https://github.com/Separius/awesome-sentence-embedding), a big list of sentence embedding resources